# Unblinding far sideband

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# USING BDT?
USEBDT = True

In [ ]:
# cale to MCC8 CV?
MCC8WEIGHTS = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = ls.fold
tree = "NeutrinoSelectionFilter"

R3BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
R3EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G_all_reco2'
R3NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
R3NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
R3DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
R3NCPI0  = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
R3CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
R3CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run3_reco2_reco2'
R3CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'
R3NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'
R3NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'

ur3mc = uproot.open(ls.ntuple_path+ls.RUN3+R3NU+ls.APPEND+".root")[fold][tree]
ur3ncpi0 = uproot.open(ls.ntuple_path+ls.RUN3+R3NCPI0+ls.APPEND+".root")[fold][tree]
ur3ccpi0 = uproot.open(ls.ntuple_path+ls.RUN3+R3CCPI0+ls.APPEND+".root")[fold][tree]
ur3nue = uproot.open(ls.ntuple_path+ls.RUN3+R3NUE+ls.APPEND+".root")[fold][tree]
ur3data = uproot.open(ls.ntuple_path+ls.RUN3+R3BNB+ls.APPEND+".root")[fold][tree]
ur3ext = uproot.open(ls.ntuple_path+ls.RUN3+R3EXT+ls.APPEND+".root")[fold][tree]
ur3dirt = uproot.open(ls.ntuple_path+ls.RUN3+R3DRT+ls.APPEND+".root")[fold][tree]
ur3lee = uproot.open(ls.ntuple_path+ls.RUN3+R3NUE+ls.APPEND+".root")[fold][tree]
ur3ccnopi = uproot.open(ls.ntuple_path+ls.RUN3+R3CCNOPI+ls.APPEND+".root")[fold][tree]
ur3cccpi = uproot.open(ls.ntuple_path+ls.RUN3+R3CCCPI+ls.APPEND+".root")[fold][tree]
ur3ncnopi = uproot.open(ls.ntuple_path+ls.RUN3+R3NCNOPI+ls.APPEND+".root")[fold][tree]
ur3nccpi = uproot.open(ls.ntuple_path+ls.RUN3+R3NCCPI+ls.APPEND+".root")[fold][tree]

R2NU = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run2_reco2_D1D2_reco2"
R2NUE = "prodgenie_bnb_intrinsic_nue_overlay_run2_v08_00_00_35_run2a_reco2_reco2"

ur2mc = uproot.open(ls.ntuple_path+ls.RUN2+R2NU+ls.APPEND+".root")[fold][tree]
ur2nue = uproot.open(ls.ntuple_path+ls.RUN2+R2NUE+ls.APPEND+".root")[fold][tree]
ur2lee = uproot.open(ls.ntuple_path+ls.RUN2+R2NUE+ls.APPEND+".root")[fold][tree]

R1BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
#R1EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C_all_reco2'
R1EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_C2_D1_D2_E1_E2_all_reco2' #Run1 + Run2
R1NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
R1NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
R1DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
R1NCPI0  = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
R1CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
R1CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run1_reco2_reco2'
R1CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
R1NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
R1NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    
ur1mc = uproot.open(ls.ntuple_path+ls.RUN1+R1NU+ls.APPEND+".root")[fold][tree]
ur1ncpi0 = uproot.open(ls.ntuple_path+ls.RUN1+R1NCPI0+ls.APPEND+".root")[fold][tree]
ur1ccpi0 = uproot.open(ls.ntuple_path+ls.RUN1+R1CCPI0+ls.APPEND+".root")[fold][tree]
ur1nue = uproot.open(ls.ntuple_path+ls.RUN1+R1NUE+ls.APPEND+".root")[fold][tree]
ur1data = uproot.open(ls.ntuple_path+ls.RUN1+R1BNB+ls.APPEND+".root")[fold][tree]
ur1ext = uproot.open(ls.ntuple_path+ls.RUN1+R1EXT+ls.APPEND+".root")[fold][tree]
ur1dirt = uproot.open(ls.ntuple_path+ls.RUN1+R1DRT+ls.APPEND+".root")[fold][tree]
ur1lee = uproot.open(ls.ntuple_path+ls.RUN1+R1NUE+ls.APPEND+".root")[fold][tree]
ur1ccnopi = uproot.open(ls.ntuple_path+ls.RUN1+R1CCNOPI+ls.APPEND+".root")[fold][tree]
ur1cccpi = uproot.open(ls.ntuple_path+ls.RUN1+R1CCCPI+ls.APPEND+".root")[fold][tree]
ur1ncnopi = uproot.open(ls.ntuple_path+ls.RUN1+R1NCNOPI+ls.APPEND+".root")[fold][tree]
ur1nccpi = uproot.open(ls.ntuple_path+ls.RUN1+R1NCCPI+ls.APPEND+".root")[fold][tree]

R123_FAR_SIDEBAND_BNB = '1enp_far_sidebands'
ur123data_far_sidebands = uproot.open(ls.ntuple_path+'data_sidebands/'+R123_FAR_SIDEBAND_BNB+".root")['nuselection'][tree]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", "trkshrhitdist0","trkshrhitdist1", #distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V"
]
#make the list unique
variables = list(set(variables))
print(variables)

variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")

WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune", "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight", "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]

r3nue = ur3nue.pandas.df(variables + WEIGHTS, flatten=False)
r3mc = ur3mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r3ncpi0 = ur3ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
r3ccpi0 = ur3ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
r3ccnopi = ur3ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
r3cccpi = ur3cccpi.pandas.df(variables + WEIGHTS, flatten=False)
r3ncnopi = ur3ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
r3nccpi = ur3nccpi.pandas.df(variables + WEIGHTS, flatten=False)
r3data = ur3data.pandas.df(variables, flatten=False)
r3ext = ur3ext.pandas.df(variables, flatten=False)
r3dirt = ur3dirt.pandas.df(variables + WEIGHTS, flatten=False)
r3lee = ur3lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

r3lee["is_signal"] = r3lee["category"] == 11
r3data["is_signal"] = r3data["category"] == 11
r3nue["is_signal"] = r3nue["category"] == 11
r3mc["is_signal"] = r3mc["category"] == 11
r3dirt["is_signal"] = r3dirt["category"] == 11
r3ext["is_signal"] = r3ext["category"] == 11
r3ncpi0["is_signal"] = r3ncpi0["category"] == 11
r3ccpi0["is_signal"] = r3ccpi0["category"] == 11
r3ccnopi["is_signal"] = r3ccnopi["category"] == 11
r3cccpi["is_signal"] = r3cccpi["category"] == 11
r3ncnopi["is_signal"] = r3ncnopi["category"] == 11
r3nccpi["is_signal"] = r3nccpi["category"] == 11
r3lee.loc[r3lee['category'] == 1, 'category'] = 111
r3lee.loc[r3lee['category'] == 10, 'category'] = 111
r3lee.loc[r3lee['category'] == 11, 'category'] = 111

uproot_v = [ur3lee,ur3mc,ur3ncpi0,ur3ccpi0,ur3ccnopi,ur3cccpi,ur3ncnopi,ur3nccpi,ur3nue,ur3ext,ur3data,ur3dirt]
df_v = [r3lee,r3mc,r3ncpi0,r3ccpi0,r3ccnopi,r3cccpi,r3ncnopi,r3nccpi,r3nue,r3ext,r3data,r3dirt]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    #shr_moliere_avg_v = up.array('shr_moliere_avg_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    #shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    #shr_moliere_avg_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_moliere_avg_v,shr_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['protonenergy'] = trk_energy_proton_sel
    #df['shrmoliereavg'] = shr_moliere_avg_sel
    
if (USEBDT == True):
    train_r3ccpi0, r3ccpi0 = train_test_split(r3ccpi0, test_size=0.5, random_state=1990)
    
r1nue = ur1nue.pandas.df(variables + WEIGHTS, flatten=False)
r1mc = ur1mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r1ncpi0 = ur1ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
r1ccpi0 = ur1ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
r1ccnopi = ur1ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
r1cccpi = ur1cccpi.pandas.df(variables + WEIGHTS, flatten=False)
r1ncnopi = ur1ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
r1nccpi = ur1nccpi.pandas.df(variables + WEIGHTS, flatten=False)
r1data = ur1data.pandas.df(variables, flatten=False)
r1ext = ur1ext.pandas.df(variables, flatten=False)
r1dirt = ur1dirt.pandas.df(variables + WEIGHTS, flatten=False)
r1lee = ur1lee.pandas.df(variables + WEIGHTSLEE, flatten=False)


r123data_far_sidebands = ur123data_far_sidebands.pandas.df(variables, flatten=False)

r1lee["is_signal"] = r1lee["category"] == 11
r1data["is_signal"] = r1data["category"] == 11
r1nue["is_signal"] = r1nue["category"] == 11
r1mc["is_signal"] = r1mc["category"] == 11
r1dirt["is_signal"] = r1dirt["category"] == 11
r1ext["is_signal"] = r1ext["category"] == 11
r1ncpi0["is_signal"] = r1ncpi0["category"] == 11
r1ccpi0["is_signal"] = r1ccpi0["category"] == 11
r1ccnopi["is_signal"] = r1ccnopi["category"] == 11
r1cccpi["is_signal"] = r1cccpi["category"] == 11
r1ncnopi["is_signal"] = r1ncnopi["category"] == 11
r1nccpi["is_signal"] = r1nccpi["category"] == 11
r1lee.loc[r1lee['category'] == 1, 'category'] = 111
r1lee.loc[r1lee['category'] == 10, 'category'] = 111
r1lee.loc[r1lee['category'] == 11, 'category'] = 111

r123data_far_sidebands["is_signal"] = r123data_far_sidebands["category"] == 11

uproot_v = [ur1lee,ur1mc,ur1ncpi0,ur1ccpi0,ur1ccnopi,ur1cccpi,ur1ncnopi,ur1nccpi,ur1nue,ur1ext,ur1data,ur1dirt, ur123data_far_sidebands]
df_v = [r1lee,r1mc,r1ncpi0,r1ccpi0,r1ccnopi,r1cccpi,r1ncnopi,r1nccpi,r1nue,r1ext,r1data,r1dirt, r123data_far_sidebands]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    #shr_moliere_avg_v = up.array('shr_moliere_avg_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    #shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    #shr_moliere_avg_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_moliere_avg_v,shr_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['protonenergy'] = trk_energy_proton_sel
    #df['shrmoliereavg'] = shr_moliere_avg_sel


r2nue = ur2nue.pandas.df(variables + WEIGHTS, flatten=False)
r2mc = ur2mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r2lee = ur2lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

r2lee["is_signal"] = r2lee["category"] == 11
r2nue["is_signal"] = r2nue["category"] == 11
r2mc["is_signal"] = r2mc["category"] == 11
r2lee.loc[r2lee['category'] == 1, 'category'] = 111
r2lee.loc[r2lee['category'] == 10, 'category'] = 111
r2lee.loc[r2lee['category'] == 11, 'category'] = 111

uproot_v = [ur2lee,ur2mc,ur2nue]
df_v = [r2lee,r2mc,r2nue]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    #shr_moliere_avg_v = up.array('shr_moliere_avg_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    #shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    #shr_moliere_avg_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_moliere_avg_v,shr_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['protonenergy'] = trk_energy_proton_sel
    #df['shrmoliereavg'] = shr_moliere_avg_sel

nue = pd.concat([r1nue,r2nue,r3nue],ignore_index=True)
#nue = pd.concat([r3nue,r1nue],ignore_index=True)
mc = pd.concat([r3mc,r2mc,r1mc],ignore_index=True)
#mc = pd.concat([r3mc,r1mc],ignore_index=True)
ncpi0 = pd.concat([r3ncpi0,r1ncpi0],ignore_index=True)
ccpi0 = pd.concat([r3ccpi0,r1ccpi0],ignore_index=True)
ccnopi = pd.concat([r3ccnopi,r1ccnopi],ignore_index=True)
cccpi = pd.concat([r3cccpi,r1cccpi],ignore_index=True)
ncnopi = pd.concat([r3ncnopi,r1ncnopi],ignore_index=True)
nccpi = pd.concat([r3nccpi,r1nccpi],ignore_index=True)
# data = pd.concat([r3data,r1data],ignore_index=True)
data = pd.concat([r123data_far_sidebands],ignore_index=True)
ext = pd.concat([r3ext,r1ext],ignore_index=True)
dirt = pd.concat([r3dirt,r1dirt],ignore_index=True)
lee = pd.concat([r1lee,r2lee,r3lee],ignore_index=True)
#lee = pd.concat([r3lee,r1lee],ignore_index=True)

In [ ]:
''' version for DAVID C
fold = ls.fold
tree = "NeutrinoSelectionFilter"

PATHDATA = '/home/david/data/searchingfornues/v08_00_00_42/cc0pinp/0420/'

R3BNB = 'neutrinoselection_filt_1enp_far_sideband_skimmed_Run3'
R3EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G_all_reco2'
R3NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
R3NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
R3DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
R3NCPI0  = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
R3CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
R3CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run3_reco2_reco2'
R3CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'
R3NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'
R3NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'

ur3mc = uproot.open(ls.ntuple_path+ls.RUN3+R3NU+ls.APPEND+".root")[fold][tree]
ur3ncpi0 = uproot.open(ls.ntuple_path+ls.RUN3+R3NCPI0+ls.APPEND+".root")[fold][tree]
ur3ccpi0 = uproot.open(ls.ntuple_path+ls.RUN3+R3CCPI0+ls.APPEND+".root")[fold][tree]
ur3nue = uproot.open(ls.ntuple_path+ls.RUN3+R3NUE+ls.APPEND+".root")[fold][tree]
ur3data = uproot.open(PATHDATA+R3BNB+".root")['nuselection'][tree]
ur3ext = uproot.open(ls.ntuple_path+ls.RUN3+R3EXT+ls.APPEND+".root")[fold][tree]
ur3dirt = uproot.open(ls.ntuple_path+ls.RUN3+R3DRT+ls.APPEND+".root")[fold][tree]
ur3lee = uproot.open(ls.ntuple_path+ls.RUN3+R3NUE+ls.APPEND+".root")[fold][tree]
ur3ccnopi = uproot.open(ls.ntuple_path+ls.RUN3+R3CCNOPI+ls.APPEND+".root")[fold][tree]
ur3cccpi = uproot.open(ls.ntuple_path+ls.RUN3+R3CCCPI+ls.APPEND+".root")[fold][tree]
ur3ncnopi = uproot.open(ls.ntuple_path+ls.RUN3+R3NCNOPI+ls.APPEND+".root")[fold][tree]
ur3nccpi = uproot.open(ls.ntuple_path+ls.RUN3+R3NCCPI+ls.APPEND+".root")[fold][tree]

R2BNB = "neutrinoselection_filt_1enp_far_sideband_skimmed_Run2"
R2NU = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run2_reco2_D1D2_reco2"
R2NUE = "prodgenie_bnb_intrinsic_nue_overlay_run2_v08_00_00_35_run2a_reco2_reco2"

ur2data = uproot.open(PATHDATA+R2BNB+".root")['nuselection'][tree]
ur2mc = uproot.open(ls.ntuple_path+ls.RUN2+R2NU+ls.APPEND+".root")[fold][tree]
ur2nue = uproot.open(ls.ntuple_path+ls.RUN2+R2NUE+ls.APPEND+".root")[fold][tree]
ur2lee = uproot.open(ls.ntuple_path+ls.RUN2+R2NUE+ls.APPEND+".root")[fold][tree]

R1BNB = 'neutrinoselection_filt_1enp_far_sideband_skimmed_C1'
#R1EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C_all_reco2'
R1EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_C2_D1_D2_E1_E2_all_reco2' #Run1 + Run2
R1NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
R1NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
R1DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
R1NCPI0  = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
R1CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
R1CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run1_reco2_reco2'
R1CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
R1NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
R1NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    
ur1mc = uproot.open(ls.ntuple_path+ls.RUN1+R1NU+ls.APPEND+".root")[fold][tree]
ur1ncpi0 = uproot.open(ls.ntuple_path+ls.RUN1+R1NCPI0+ls.APPEND+".root")[fold][tree]
ur1ccpi0 = uproot.open(ls.ntuple_path+ls.RUN1+R1CCPI0+ls.APPEND+".root")[fold][tree]
ur1nue = uproot.open(ls.ntuple_path+ls.RUN1+R1NUE+ls.APPEND+".root")[fold][tree]
ur1data = uproot.open(PATHDATA+R1BNB+".root")['nuselection'][tree]
ur1ext = uproot.open(ls.ntuple_path+ls.RUN1+R1EXT+ls.APPEND+".root")[fold][tree]
ur1dirt = uproot.open(ls.ntuple_path+ls.RUN1+R1DRT+ls.APPEND+".root")[fold][tree]
ur1lee = uproot.open(ls.ntuple_path+ls.RUN1+R1NUE+ls.APPEND+".root")[fold][tree]
ur1ccnopi = uproot.open(ls.ntuple_path+ls.RUN1+R1CCNOPI+ls.APPEND+".root")[fold][tree]
ur1cccpi = uproot.open(ls.ntuple_path+ls.RUN1+R1CCCPI+ls.APPEND+".root")[fold][tree]
ur1ncnopi = uproot.open(ls.ntuple_path+ls.RUN1+R1NCNOPI+ls.APPEND+".root")[fold][tree]
ur1nccpi = uproot.open(ls.ntuple_path+ls.RUN1+R1NCCPI+ls.APPEND+".root")[fold][tree]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V"
]
#make the list unique
variables = list(set(variables))
print(variables)

variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")

WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune", "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight", "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]

r3nue = ur3nue.pandas.df(variables + WEIGHTS, flatten=False)
r3mc = ur3mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r3ncpi0 = ur3ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
r3ccpi0 = ur3ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
r3ccnopi = ur3ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
r3cccpi = ur3cccpi.pandas.df(variables + WEIGHTS, flatten=False)
r3ncnopi = ur3ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
r3nccpi = ur3nccpi.pandas.df(variables + WEIGHTS, flatten=False)
r3data = ur3data.pandas.df(variables, flatten=False)
r3ext = ur3ext.pandas.df(variables, flatten=False)
r3dirt = ur3dirt.pandas.df(variables + WEIGHTS, flatten=False)
r3lee = ur3lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

r3lee["is_signal"] = r3lee["category"] == 11
r3data["is_signal"] = r3data["category"] == 11
r3nue["is_signal"] = r3nue["category"] == 11
r3mc["is_signal"] = r3mc["category"] == 11
r3dirt["is_signal"] = r3dirt["category"] == 11
r3ext["is_signal"] = r3ext["category"] == 11
r3ncpi0["is_signal"] = r3ncpi0["category"] == 11
r3ccpi0["is_signal"] = r3ccpi0["category"] == 11
r3ccnopi["is_signal"] = r3ccnopi["category"] == 11
r3cccpi["is_signal"] = r3cccpi["category"] == 11
r3ncnopi["is_signal"] = r3ncnopi["category"] == 11
r3nccpi["is_signal"] = r3nccpi["category"] == 11
r3lee.loc[r3lee['category'] == 1, 'category'] = 111
r3lee.loc[r3lee['category'] == 10, 'category'] = 111
r3lee.loc[r3lee['category'] == 11, 'category'] = 111

uproot_v = [ur3lee,ur3mc,ur3ncpi0,ur3ccpi0,ur3ccnopi,ur3cccpi,ur3ncnopi,ur3nccpi,ur3nue,ur3ext,ur3data,ur3dirt]
df_v = [r3lee,r3mc,r3ncpi0,r3ccpi0,r3ccnopi,r3cccpi,r3ncnopi,r3nccpi,r3nue,r3ext,r3data,r3dirt]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel

if (USEBDT == True):
    train_r3ccpi0, r3ccpi0 = train_test_split(r3ccpi0, test_size=0.5, random_state=1990)
    
r1nue = ur1nue.pandas.df(variables + WEIGHTS, flatten=False)
r1mc = ur1mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r1ncpi0 = ur1ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
r1ccpi0 = ur1ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
r1ccnopi = ur1ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
r1cccpi = ur1cccpi.pandas.df(variables + WEIGHTS, flatten=False)
r1ncnopi = ur1ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
r1nccpi = ur1nccpi.pandas.df(variables + WEIGHTS, flatten=False)
r1data = ur1data.pandas.df(variables, flatten=False)
r1ext = ur1ext.pandas.df(variables, flatten=False)
r1dirt = ur1dirt.pandas.df(variables + WEIGHTS, flatten=False)
r1lee = ur1lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

r1lee["is_signal"] = r1lee["category"] == 11
r1data["is_signal"] = r1data["category"] == 11
r1nue["is_signal"] = r1nue["category"] == 11
r1mc["is_signal"] = r1mc["category"] == 11
r1dirt["is_signal"] = r1dirt["category"] == 11
r1ext["is_signal"] = r1ext["category"] == 11
r1ncpi0["is_signal"] = r1ncpi0["category"] == 11
r1ccpi0["is_signal"] = r1ccpi0["category"] == 11
r1ccnopi["is_signal"] = r1ccnopi["category"] == 11
r1cccpi["is_signal"] = r1cccpi["category"] == 11
r1ncnopi["is_signal"] = r1ncnopi["category"] == 11
r1nccpi["is_signal"] = r1nccpi["category"] == 11
r1lee.loc[r1lee['category'] == 1, 'category'] = 111
r1lee.loc[r1lee['category'] == 10, 'category'] = 111
r1lee.loc[r1lee['category'] == 11, 'category'] = 111

uproot_v = [ur1lee,ur1mc,ur1ncpi0,ur1ccpi0,ur1ccnopi,ur1cccpi,ur1ncnopi,ur1nccpi,ur1nue,ur1ext,ur1data,ur1dirt]
df_v = [r1lee,r1mc,r1ncpi0,r1ccpi0,r1ccnopi,r1cccpi,r1ncnopi,r1nccpi,r1nue,r1ext,r1data,r1dirt]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel


r2data = ur2data.pandas.df(variables, flatten=False)
r2nue = ur2nue.pandas.df(variables + WEIGHTS, flatten=False)
r2mc = ur2mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
r2lee = ur2lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

r2lee["is_signal"] = r2lee["category"] == 11
r2nue["is_signal"] = r2nue["category"] == 11
r2mc["is_signal"] = r2mc["category"] == 11
r2lee.loc[r2lee['category'] == 1, 'category'] = 111
r2lee.loc[r2lee['category'] == 10, 'category'] = 111
r2lee.loc[r2lee['category'] == 11, 'category'] = 111

uproot_v = [ur2data,ur2lee,ur2mc,ur2nue]
df_v = [r2data,r2lee,r2mc,r2nue]
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    
    
    
nue = pd.concat([r1nue,r2nue,r3nue],ignore_index=True)
#nue = pd.concat([r3nue,r1nue],ignore_index=True)
mc = pd.concat([r3mc,r2mc,r1mc],ignore_index=True)
#mc = pd.concat([r3mc,r1mc],ignore_index=True)
ncpi0 = pd.concat([r3ncpi0,r1ncpi0],ignore_index=True)
ccpi0 = pd.concat([r3ccpi0,r1ccpi0],ignore_index=True)
ccnopi = pd.concat([r3ccnopi,r1ccnopi],ignore_index=True)
cccpi = pd.concat([r3cccpi,r1cccpi],ignore_index=True)
ncnopi = pd.concat([r3ncnopi,r1ncnopi],ignore_index=True)
nccpi = pd.concat([r3nccpi,r1nccpi],ignore_index=True)
data = pd.concat([r3data,r2data,r1data],ignore_index=True)
ext = pd.concat([r3ext,r1ext],ignore_index=True)
dirt = pd.concat([r3dirt,r1dirt],ignore_index=True)
lee = pd.concat([r1lee,r2lee,r3lee],ignore_index=True)
#lee = pd.concat([r3lee,r1lee],ignore_index=True)
'''

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,dirt]

for i,df in enumerate(df_v):
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] = df['weightSpline'] * df['weightTune'] * 0.759
    #df['weightSpline']  = df['weightSpline']  * df['weightTune']

In [ ]:
# adjust from MCC9 CV to MCC8 CV

if (MCC8WEIGHTS == True):

    # scaling for QE
    CV_bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,5.0]
    CV_scaling = [2.5,2.0,1.7,1.45,1.3,1.25,1.175,1.15,1.14,1.1]
    # scaling for RES

    mc.loc[ (mc['interaction'] == 10), 'weightSpline' ] = 2 * mc['weightSpline']
    ncpi0.loc[ (ncpi0['interaction'] == 10), 'weightSpline' ] = 2 * ncpi0['weightSpline']
    ccpi0.loc[ (ccpi0['interaction'] == 10), 'weightSpline' ] = 2 * ccpi0['weightSpline']
    ccnopi.loc[ (ccnopi['interaction'] == 10), 'weightSpline' ] = 2 * ccnopi['weightSpline']
    cccpi.loc[ (cccpi['interaction'] == 10), 'weightSpline' ] = 2 * cccpi['weightSpline']
    ncnopi.loc[ (ncnopi['interaction'] == 10), 'weightSpline' ] = 2 * ncnopi['weightSpline']
    nccpi.loc[ (nccpi['interaction'] == 10), 'weightSpline' ] = 2 * nccpi['weightSpline']
    nue.loc[ (nue['interaction'] == 10), 'weightSpline' ] = 2 * nue['weightSpline']
    lee.loc[ (lee['interaction'] == 10), 'weightSpline' ] = 2 * lee['weightSpline']
    dirt.loc[ (dirt['interaction'] == 10), 'weightSpline' ] = 2 * dirt['weightSpline']

    for i, CV_bin in enumerate(CV_bins):

        if i == 0:
            continue

        mc.loc[ (mc['nu_e'] > CV_bins[i-1]) & (mc['nu_e'] < CV_bins[i]) & (mc['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * mc['weightSpline']
        nue.loc[ (nue['nu_e'] > CV_bins[i-1]) & (nue['nu_e'] < CV_bins[i]) & (nue['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * nue['weightSpline']
        ncpi0.loc[ (nc['nu_e'] > CV_bins[i-1]) & (ncpi0['nu_e'] < CV_bins[i]) & (ncpi0['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ncpi0['weightSpline']
        ccpi0.loc[ (ccpi0['nu_e'] > CV_bins[i-1]) & (ccpi0['nu_e'] < CV_bins[i]) & (ccpi0['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ccpi0['weightSpline']
        ccnopi.loc[ (ccnopi['nu_e'] > CV_bins[i-1]) & (ccnopi['nu_e'] < CV_bins[i]) & (ccnopi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ccnopi['weightSpline']
        cccpi.loc[ (cccpi['nu_e'] > CV_bins[i-1]) & (cccpi['nu_e'] < CV_bins[i]) & (cccpi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * cccpi['weightSpline']
        ncnopi.loc[ (ncnopi['nu_e'] > CV_bins[i-1]) & (ncnopi['nu_e'] < CV_bins[i]) & (ncnopi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ncnopi['weightSpline']
        nccpi.loc[ (nccpi['nu_e'] > CV_bins[i-1]) & (nccpi['nu_e'] < CV_bins[i]) & (nccpi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * nccpi['weightSpline']
        lee.loc[ (lee['nu_e'] > CV_bins[i-1]) & (lee['nu_e'] < CV_bins[i]) & (lee['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * lee['weightSpline']
        dirt.loc[ (dirt['nu_e'] > CV_bins[i-1]) & (dirt['nu_e'] < CV_bins[i]) & (dirt['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * dirt['weightSpline']



In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    # and the 2d angle difference
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    #df['anglediff_V'] = np.abs(df['secondshower_V_dir']-df['shrclusdir1'])
    #df['anglediff_U'] = np.abs(df['secondshower_U_dir']-df['shrclusdir0'])


In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    df['shr_tkfit_dedx_avg'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])
    df['shr_tkfit_2cm_dedx_avg'] = (df['shr_tkfit_2cm_nhits_Y']*df['shr_tkfit_2cm_dedx_Y'] + df['shr_tkfit_2cm_nhits_U']*df['shr_tkfit_2cm_dedx_U'] + df['shr_tkfit_2cm_nhits_V']*df['shr_tkfit_2cm_dedx_V'])/df['shr_tkfit_2cm_nhits_tot']
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])
    df['shr_tkfit_gap10_dedx_avg'] = (df['shr_tkfit_gap10_nhits_Y']*df['shr_tkfit_gap10_dedx_Y'] + df['shr_tkfit_gap10_nhits_U']*df['shr_tkfit_gap10_dedx_U'] + df['shr_tkfit_gap10_nhits_V']*df['shr_tkfit_gap10_dedx_V'])/df['shr_tkfit_gap10_nhits_tot']
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']

In [ ]:
INTERCEPT = 0.0
SLOPE = 0.83

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df["reco_e_qe"] = 0.938*((df["shr_energy"]+INTERCEPT)/SLOPE)/(0.938 - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
    df["extdata"] = np.zeros_like(df["shr_energy"])
data["bnbdata"] = np.ones_like(data["shr_energy"])
ext["extdata"] = np.ones_like(ext["shr_energy"])

In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
# not needed anymore since we improved matching with filtered samples
#ncpi0 = ncpi0.query('category != 5')
#ccpi0 = ccpi0.query('category != 5')
#ccnopi = ccnopi.query('category != 5')
#nccpi = nccpi.query('category != 5')
#ncnopi = ncnopi.query('category != 5')

## avoid recycling unbiased ext events (i.e. selecting a slice with little nu content from these samples)
ccnopi = ccnopi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
cccpi = cccpi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
ncnopi = ncnopi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
nccpi = nccpi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')

# add back the cosmic category, for background only
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
# variables to be trained on
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

In [ ]:
LABELS =  ['pi0','nonpi0']
#LABELS =  ["bkg"]

if (USEBDT == True):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            mc[label+"_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nue[label+"_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ext[label+"_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            data[label+"_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            lee[label+"_score"] = booster.predict(
                xgb.DMatrix(lee[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ncpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ccpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ccnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            cccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(cccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ncnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(nccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    "ncpi0": ncpi0,
    "ccpi0": ccpi0,
    "ccnopi": ccnopi,
    "cccpi": cccpi,
    "ncnopi": ncnopi,
    "nccpi": nccpi,
    "lee": lee
}

#scaling = 101.0/4.21 #0218
# scaling = 101.0/4.84 #0304
#scaling = 69.6/4.84 #0304
#scaling = 125.0/4.84 #0304
scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 1.48

#''' 0304
weights = {
    "mc": 1.61e-01 * scaling, 
    "ext": 5.01e-01 * scaling, 
    "nue": 3.32e-03 * scaling,
    "lee": 3.32e-03 * scaling,
    "dirt": 9.09e-01 * scaling,
    "ncpi0": 1.19e-01 * scaling,
    "ccpi0": 5.92e-02 * SPLIT * scaling,
    "ncnopi": 5.60e-02 * scaling,
    "nccpi": 2.58e-02 * scaling,
    "ccnopi": 6.48e-02 * scaling,
    "cccpi": 5.18e-02 * scaling,
}
pot = 5.88e20*scaling

my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

# Start Plotting

In [ ]:
from unblinding_far_sideband import *
from scipy.stats import poisson


#studies showing agreement between data and MC stats (w/ confidence intervals) vs. cut value

In [ ]:
# studies showing agreement between data and MC stats (w/ confidence intervals) vs. cut value

# thruth-cut on MC sample to avoid double counting with truth-filters (i.e. ccpi0, ncpi0, cccpi, ...)
NU_Q = "~(abs(nu_pdg) == 12 & ccnc == 0)"
NU_Q += " & ~(mcf_np0==1 & mcf_nmp==0 & mcf_nmm==0 & mcf_nem==0 & mcf_nep==0)"
NU_Q += " & ~(mcf_pass_ccpi0==1)"
NU_Q += " & ~(mcf_pass_ccnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_ncnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_cccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_nccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"

def GetNumEvents(VAR,EMIN,EMAX,QUERY):

    TOT  = 0.
    DATA = 0.
    
    for key, sample in samples.items():
        THISQUERY = QUERY
        # calculate weight for sample
        #print (THISQUERY)
        weight = 1
        if (key != 'data'):
            weight = weights[key]    
        if (key == 'mc'):
            THISQUERY += ' and %s'%NU_Q
        # calculate bare entries
        dfsub = sample.query(THISQUERY)
        vals_v = dfsub[VAR].values
        weights_v = np.ones(len(vals_v)) * weight
        if ((key != 'data') and (key != 'ext')):
            weights_v = dfsub['weightSplineTimesTune'].values * weight
        v,be = np.histogram(vals_v,bins=np.array([EMIN,EMAX]),weights=weights_v)
        
        #print ('sample %s has %i entries'%(key,v[0]))

        if (key == 'data'):
            DATA += v[0]
        else:
            TOT += v[0]
            
    #print ('DATA : %.01f. EXPECTATION : %.01f'%(DATA,TOT))
    # poisson interval
    # 68%
    range68 = poisson.interval(0.68,TOT)
    range95 = poisson.interval(0.95,TOT)
    range99 = poisson.interval(0.99,TOT)
    #print ('range 68%% %.02f -- %.02f'%(range68[0],range68[1]))
    #print ('range 95%% %.02f -- %.02f'%(range95[0],range95[1]))
    return DATA,TOT,range68[0],range68[1],range95[0],range95[1],range99[0],range99[1]

BDTCUT_V = np.concatenate((np.linspace(0.0,0.5,21),np.linspace(0.5,1.0,15)),axis=0)
#BDTCU
DATA_V = []
EXP_V = []
MIN68_V = []
MAX68_V = []
MIN95_V = []
MAX95_V = []
MIN99_V = []
MAX99_V = []

#GetNumEvents('reco_e',0.05,0.85,NPPRESQ+' and pi0_score > 0.7 and category!=111 and n_showers_contained == 1')

#'''
for BDTCUT in BDTCUT_V:
    #print ('BDT cut : %.02f'%BDTCUT)
    Q = stages_queries[1]
    #print (Q)
    Q += ' and nonpi0_score > %.02f and pi0_score < 1.0 and category != 111'%BDTCUT
    d,e,m68,M68,m95,M95,m99,M99 = GetNumEvents('reco_e',1.05,2.05,Q)
    DATA_V.append(d)
    EXP_V.append(e)
    MIN68_V.append(m68)
    MAX68_V.append(M68)
    MIN95_V.append(m95)
    MAX95_V.append(M95)
    MIN99_V.append(m99)
    MAX99_V.append(M99)
plt.figure(figsize=(10,6))

plt.fill_between(BDTCUT_V,MIN99_V,MAX99_V,alpha=0.2,color='c',label='99% interval')
plt.fill_between(BDTCUT_V,MIN95_V,MAX95_V,alpha=0.2,color='b',label='95% interval')
plt.fill_between(BDTCUT_V,MIN68_V,MAX68_V,alpha=0.2,color='m',label='68% interval')

plt.plot(BDTCUT_V,DATA_V,color='r',lw=2,label='observation')
plt.plot(BDTCUT_V,EXP_V,color='b',lw=2,label='expectation')

plt.grid()
plt.xlabel('$\pi^0$ BDT score cut')
plt.ylabel('surviving events [%.02fE20 POT]'%(pot/1e20))
plt.gca().ticklabel_format(scilimits=(-2,4))
plt.yscale('log')
plt.legend(loc=1,fontsize=14)
plt.show()
#'''

# BDT scan

In [ ]:
stage = 3
no_data = False
no_leg = True
bins_reduction_factor = {
    1: 1.0,
    2: 0.5,
    3: 0.5,
    4: 0.5,
    5: 0.5,
}

In [ ]:
for bdtcut in bdt_scan:
    
    print ('BDT cut value of %.01f'%bdtcut)
    
    if (bdtcut == 0.0):
        break
    for which_variables in plot_variables.keys():
        this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
        if no_data:
            this_folder += 'dry_run/bdtscan%02i/'%(int(bdtcut*10))
        else:
            this_folder += 'with_data/bdtscan%02i/'%(int(bdtcut*10))
        this_folder += 'stage_{}/'.format(stage)
        this_folder += (which_variables + '/')
        !mkdir -p $this_folder

        this_query = stages_queries[stage]
        this_title = stages_titles[stage]
        this_title += '\n nonpi0_bdt > %.01f and Cosmic3D cuts'%bdtcut
        
        print (this_title)
        
        this_query += ' and CosmicDirAll3D > -0.98 and CosmicDirAll3D < 0.98'
        this_query += ' and CosmicIPAll3D > 30'
        this_query += ' and nonpi0_score > %.01f'%bdtcut

        if no_data:
            this_query += ' and bnbdata==0'

        #print(which_variables)
        this_plot_variables = plot_variables[which_variables]

        for VARIABLE, BINS, RANGE, XTIT in this_plot_variables:
            #print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = my_plotter.plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=False,
                stacksort=3,
                title=XTIT,
                bins=int(BINS*bins_reduction_factor[stage]),
                range=RANGE,
            )[0:3]
            if 'score' in VARIABLE:
                ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)
            ax1.set_title(this_title, loc='left')
            plt.tight_layout()
            fig.savefig(this_folder+VARIABLE+'.png', dpi=250)    
    #         fig.savefig(this_folder+VARIABLE+'.pdf')
            plt.close()

## Plot variables

In [ ]:
plot_variables.keys()

In [ ]:
variables_set = []
#variables_set.append('insensitive_variables')
#variables_set.append('input_bdt')
#variables_set.append('bdt_scores')
#variables_set.append('vlvars')
variables_set.append('all')

In [ ]:
print(stage)

In [ ]:
for which_variables in variables_set:
    this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
    if no_data:
        this_folder += 'dry_run/'
    else:
        this_folder += 'with_data/'
    this_folder += 'stage_{}/'.format(stage)
    this_folder += (which_variables + '/')
    !mkdir -p $this_folder
    
    this_query = stages_queries[stage]
    this_title = stages_titles[stage]

    if no_data:
        this_query += ' and bnbdata==0'
    
    print(which_variables)
    this_plot_variables = plot_variables[which_variables]

    for VARIABLE, BINS, RANGE, XTIT in this_plot_variables:
        print(VARIABLE, BINS, RANGE, XTIT)
        fig, ax1, ax2 = my_plotter.plot_variable(
            VARIABLE,   
            query=this_query,
            kind="event_category",
            draw_sys=True,
            stacksort=3,
            title=XTIT,
            bins=BINS,#*bins_reduction_factor[stage],
            range=RANGE,
        )[0:3]
        if 'score' in VARIABLE:
            ax1.set_yscale('log')
        else:
            ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)
        ax1.set_title(this_title, loc='left')
        plt.tight_layout()
        fig.savefig(this_folder+VARIABLE+'.png', dpi=250)    
#         fig.savefig(this_folder+VARIABLE+'.pdf')
        plt.close()

In [ ]:
data.query(HIGH_ENERGY+' and '+NPPRESEQ_one_shower_one_track+' and '+BDTCQ)[['run','sub','evt','reco_e','secondshower_Y_nhit','shrmoliereavg','shr_tkfit_nhits_tot','trk_score','trkshrhitdist0','trkshrhitdist1','trkshrhitdist2','shr_theta','subcluster','trk_len','trkpid']]

# All stage together

In [ ]:
plot_variables.keys()

In [ ]:
variables_set = []
variables_set.append('insensitive_variables')
variables_set.append('input_bdt')
variables_set.append('bdt_scores')

In [ ]:
no_data = False
bins_reduction_factor = {
    1: 1,
    2: 1,
    3: 0.5,
    4: 0.5,
    5: 0.5,
}

In [ ]:
for stage in range(1, 6):
    for which_variables in plot_variables.keys():
        this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
        if no_data:
            this_folder += 'dry_run/'
        else:
            this_folder += 'with_data/'
        this_folder += 'stage_{}/'.format(stage)
        this_folder += (which_variables + '/')
        !mkdir -p $this_folder

        this_query = stages_queries[stage]
        this_title = stages_titles[stage]

        if no_data:
            this_query += ' and bnbdata==0'

        print(which_variables)
        this_plot_variables = plot_variables[which_variables]

        for VARIABLE, BINS, RANGE, XTIT in this_plot_variables:
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = my_plotter.plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=False,
                stacksort=3,
                title=XTIT,
                bins=int(BINS*bins_reduction_factor[stage]),
                range=RANGE,
            )[0:3]
            if 'score' in VARIABLE:
                ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)
            ax1.set_title(this_title, loc='left')
            plt.tight_layout()
            fig.savefig(this_folder+VARIABLE+'.png', dpi=250)    
    #         fig.savefig(this_folder+VARIABLE+'.pdf')
            plt.close()

# All stage together with pi0 scaling

In [ ]:
from unblinding_far_sideband import *

In [ ]:
plot_variables.keys()

In [ ]:
variables_set = []
# variables_set.append('insensitive_variables')
# variables_set.append('input_bdt')
variables_set.append('bdt_scores')

In [ ]:
no_data = False
no_leg = False
bins_reduction_factor = {
    1: 1,
    2: 1,
    3: 0.5,
    4: 2,
    5: 0.5,
}

In [ ]:
for stage in [1]:
    for which_variables in variables_set:
        this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
        this_folder += 'with_pi0_scaling/'
        this_folder += 'stage_{}/'.format(stage)
        this_folder += (which_variables + '/')
        !mkdir -p $this_folder

        this_query = stages_queries[stage]
        this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759'

        if no_data:
            this_query += ' and bnbdata==0'

        print(which_variables)
        this_plot_variables = plot_variables[which_variables]

        for VARIABLE, BINS, RANGE, XTIT in this_plot_variables:
            print(VARIABLE, BINS, RANGE, XTIT)
            out = my_plotter.plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=int(BINS*bins_reduction_factor[stage]),
                range=RANGE,
            )
            fig, ax1, ax2 = out[0:3]
            
            if no_leg:
                ax1.legend().set_visible(False) 
            else:
                ax1.set_title(this_title, loc='left')
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)
        
            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if no_leg:
                save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
    #         fig.savefig(save_path + '.pdf')    
#             plt.close()

# Plots with systematic uncertainties

In [ ]:
#dedx after very loose box cuts
detsys = {
    'nue': [       0, 0.79671758, 0.34827979, 0.321645,   0.40333879, 0.1853362,
 0.24609908, 0.33585692, 0.26870701, 0.40225343, 0.50266717, 0.65527298,
 1.24068308, 0.59963333, 0.73699676]
}

stage = 2
no_leg = True
this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
this_folder += 'with_pi0_scaling/'
this_folder += 'stage_{}/'.format(stage)
this_folder += (which_variables + '/')
!mkdir -p $this_folder

this_query = stages_queries[stage]
this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759' + '\nwith detector systematics'

this_plot_variables = plot_variables[which_variables]

VARIABLE, BINS, RANGE, XTIT = 'shr_tkfit_dedx_max',15,(0,10),"shr tkfit dE/dx (max, 0-4 cm) [MeV/cm]"

out = my_plotter.plot_variable(
    VARIABLE,   
    query=this_query,
    kind="event_category",
    draw_sys=True,
    detsys=detsys,
    stacksort=3,
    title=XTIT,
    bins=int(BINS*bins_reduction_factor[stage]),
    range=RANGE,
)
fig, ax1, ax2 = out[0:3]

if no_leg:
    ax1.legend().set_visible(False) 
else:
    ax1.set_title(this_title, loc='left')
    ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

plt.tight_layout()
save_path = this_folder+VARIABLE
if no_leg:
    save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
#         fig.savefig(save_path + '.pdf')    
#             plt.close()

In [ ]:
#pi0 score after very loose box cuts
detsys = {
    'nue': np.array([0.14096838, 0.52125645, 0.30698031, 0.16230782, 0.20707762, 0.20503962,
 0.4905541,  0.17894124, 0.21784577, 0.12043394])

}

stage = 2
no_leg = True
this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
this_folder += 'with_pi0_scaling/'
this_folder += 'stage_{}/'.format(stage)
this_folder += (which_variables + '/')
!mkdir -p $this_folder

this_query = stages_queries[stage]
this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759' + '\nwith detector systematics'

this_plot_variables = plot_variables[which_variables]

VARIABLE, BINS, RANGE, XTIT = 'pi0_score',10,(0., 1),"BDT $\pi^0$ score"

out = my_plotter.plot_variable(
    VARIABLE,   
    query=this_query,
    kind="event_category",
    draw_sys=True,
    detsys=detsys,
    stacksort=3,
    title=XTIT,
    bins=int(BINS*bins_reduction_factor[stage]),
    range=RANGE,
)
fig, ax1, ax2 = out[0:3]

if no_leg:
    ax1.legend().set_visible(False) 
else:
    ax1.set_title(this_title, loc='left')
    ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

plt.tight_layout()
save_path = this_folder+VARIABLE
if no_leg:
    save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
#         fig.savefig(save_path + '.pdf')    
#             plt.close()

In [ ]:
#dedx after far energy
detsys = {
    'nue': [       0.0, 0.91611439, 0.59984853, 0.36909447, 0.33792165, 0.28304237,
 0.33751168, 0.22135163, 0.29769619, 0.15383638, 0.23421376, 0.22867616,
 0.37325692, 0.18749145, 0.19447481, 0.27023232, 0.30524752, 0.31487403,
 0.3269046,  0.41923414, 0.31404011, 0.49550582, 0.69728428, 0.594751,
 0.52312608]

}

stage = 1
no_leg = True
# this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
# this_folder += 'with_pi0_scaling/'
# this_folder += 'stage_{}/'.format(stage)
# !mkdir -p $this_folder

this_query = stages_queries[stage]
this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759' + '\nwith detector systematics'

VARIABLE, BINS, RANGE, XTIT = 'shr_tkfit_dedx_max',25,(0,10),"shr tkfit dE/dx (max, 0-4 cm) [MeV/cm]"

out = my_plotter.plot_variable(
    VARIABLE,   
    query=this_query,
    kind="event_category",
    draw_sys=True,
    detsys=detsys,
    stacksort=3,
    title=XTIT,
    bins=int(BINS*bins_reduction_factor[stage]),
    range=RANGE,
)
fig, ax1, ax2 = out[0:3]

if no_leg:
    ax1.legend().set_visible(False) 
else:
    ax1.set_title(this_title, loc='left')
    ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

plt.tight_layout()
save_path = this_folder+VARIABLE
if no_leg:
    save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
#         fig.savefig(save_path + '.pdf')    
#             plt.close()

In [ ]:
#dedx after far energy
detsys = {
    'nue': [0.09027234, 0.08413832, 0.07560043, 0.09114168, 0.05486566, 0.2446849,
 0.23054174, 0.46507027, 0.20827211, 0.45844788, 0.39172489, 0.40934926,
 0.49517035, 0.77600138, 0.88587843, 1.43771866, 1.27976322, 0.86024124,
 1.15451296, 1.09811053]


}

stage = 1
no_leg = True
# this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
# this_folder += 'with_pi0_scaling/'
# this_folder += 'stage_{}/'.format(stage)
# !mkdir -p $this_folder

this_query = stages_queries[stage]
this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759' + '\nwith detector systematics'

VARIABLE, BINS, RANGE, XTIT = 'shrmoliereavg',20,(0,50),"average Moliere angle [degrees]"

out = my_plotter.plot_variable(
    VARIABLE,   
    query=this_query,
    kind="event_category",
    draw_sys=True,
    detsys=detsys,
    stacksort=3,
    title=XTIT,
    bins=int(BINS*bins_reduction_factor[stage]),
    range=RANGE,
)
fig, ax1, ax2 = out[0:3]

if no_leg:
    ax1.legend().set_visible(False) 
else:
    ax1.set_title(this_title, loc='left')
    ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

plt.tight_layout()
save_path = this_folder+VARIABLE
if no_leg:
    save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
#         fig.savefig(save_path + '.pdf')    
#             plt.close()

In [ ]:
#dedx after far energy
detsys = {
    'nue': [0.05403331, 0.12645447, 0.12424552, 0.12682673, 0.08274685, 0.11693988,
0.27228293, 0.13337103, 0.19497178, 0.1070512,
]


}

stage = 1
no_leg = True
# this_folder = ls.plots_path+'PELEE_far_sideband_unblinding/'
# this_folder += 'with_pi0_scaling/'
# this_folder += 'stage_{}/'.format(stage)
# !mkdir -p $this_folder

this_query = stages_queries[stage]
this_title = stages_titles[stage] + '\n' + r'$\pi^0$ scaled by 0.759' + '\nwith detector systematics'

VARIABLE, BINS, RANGE, XTIT = 'pi0_score',10,(0., 1),"BDT $\pi^0$ score"

out = my_plotter.plot_variable(
    VARIABLE,   
    query=this_query,
    kind="event_category",
    draw_sys=True,
    detsys=detsys,
    stacksort=3,
    title=XTIT,
    bins=int(BINS*bins_reduction_factor[stage]),
    range=RANGE,
)
fig, ax1, ax2 = out[0:3]

if no_leg:
    ax1.legend().set_visible(False) 
else:
    ax1.set_title(this_title, loc='left')
    ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

ax1.set_yscale('log')
plt.tight_layout()
save_path = this_folder+VARIABLE
if no_leg:
    save_path += '_noleg'
#             fig.savefig(save_path + '.png', dpi=250)    
#         fig.savefig(save_path + '.pdf')    
#             plt.close()